In [1]:
import cv2
import numpy as np

In [2]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

In [3]:
def jacob_warp_affine(x,y):
    return np.array([[x,0,y,0,1,0],[0,x,0,y,0,1]],dtype=np.float64) ##(2,3)

In [4]:
def linearize_warp(wf):
    assert (wf.shape==(2,3))
    return np.float64([[wf[0,0]-1.0],[wf[1,0]],[wf[0,1]],[wf[1,1]-1.0],[wf[0,2]],[wf[1,2]]]) ##(6,1)

In [5]:
def make_warp(p):
    assert (p.shape==(6,1))
    return np.float64([[p[0,0]+1.0,p[2,0],p[4,0]],[p[1,0],p[3,0]+1.0,p[5,0]]]) ##(2,3)

In [6]:
def inverse_param(p):
    assert (p.shape==(6,1))
    denom = ((1+p[0,0])*(1+p[3,0]) - (p[1,0]*p[2,0]))
    invp = np.zeros(p.shape,dtype=np.float64)
    invp[0,0] = (-p[0,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[1,0] = (-p[1,0])
    invp[2,0] = (-p[2,0])
    invp[3,0] = (-p[3,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[4,0] = (-p[4,0])-(p[3,0]*p[4,0])+(p[2,0]*p[5,0])
#     invp[5,0] = (-p[5,0])-(p[0,0]*p[5,0])+(p[1,0]*p[2,0]) ## THIS IS TECHNICALLY WRONG
    invp[5,0] = (-p[5,0])-(p[0,0]*p[5,0])+(p[1,0]*p[4,0]) ## THIS IS PRACTICALLY WRONG
    invp = invp/denom
    return invp ##(6,1)

In [7]:
def compose_param(p1,p2):
    assert (p1.shape==(6,1))
    assert (p2.shape==(6,1))
    comp = np.zeros(p1.shape,dtype=np.float64)
    comp[0,0] = p1[0,0]+p2[0,0]+(p1[0,0]*p2[0,0])+(p1[2,0]*p2[1,0])
    comp[1,0] = p1[1,0]+p2[1,0]+(p1[1,0]*p2[0,0])+(p1[3,0]*p2[1,0])
    comp[2,0] = p1[2,0]+p2[2,0]+(p1[0,0]*p2[2,0])+(p1[2,0]*p2[3,0])
    comp[3,0] = p1[3,0]+p2[3,0]+(p1[1,0]*p2[2,0])+(p1[3,0]*p2[3,0])
    comp[4,0] = p1[4,0]+p2[4,0]+(p1[0,0]*p2[4,0])+(p1[2,0]*p2[5,0])
    comp[5,0] = p1[5,0]+p2[5,0]+(p1[1,0]*p2[4,0])+(p1[3,0]*p2[5,0])
    return comp ##(6,1)

In [23]:
def gaussNewton(img,template,warp,epsilon,transform):
    template = template.astype('float64')
    img = img.astype('float64')
    assert (len(img.shape)==2), "Grayscale image should be given"
    ## template is the target image
    ## img is initial image
    ## warp is the initial guess warp ([1+p1,p3,p5],[p2,1+p4,p6])
    ## transform is one of "AFFINE" "TRANSLATIONAL"
    ## epsilon defines the ending condition
    

    template_shape = template.shape
    tmplx,tmply = template_shape
    
    ## Calculating gradients (3)
    
    # kerX = np.float64([[-0.5,0.0,0.5]])
    # kerY = np.float64([-0.5,0.0,0.5])
    # gradX = cv2.filter2D(template,-1,kerX)
    # gradY = cv2.filter2D(template,-1,kerY)
    
    gradTx = cv2.Sobel(template,cv2.CV_64F,1,0,ksize=3) ##(x,y)
    gradTy = cv2.Sobel(template,cv2.CV_64F,0,1,ksize=3) ##(x,y)
    grad = np.stack([gradTx,gradTy],axis=-1) ##(x,y,2)
    grad = np.expand_dims(grad,axis=2) ##(x,y,1,2)
    
    ## Calculating jacobian of template image (4)
    jacob_tmpl = np.zeros([tmplx,tmply,2,6]) ## (x,y,2,6)
    for i in range(tmplx):
        for j in range(tmply):
            # because i,j in numpy image correspond to j,i in image axis
            jacob_tmpl[i,j] = jacob_warp_affine(j,i)
    
    ## Calculating steepest descent (5)
    steep_desc = np.matmul(grad,jacob_tmpl) ##(x,y,1,6)
    steep_desc_trans = np.transpose(steep_desc,[0,1,3,2]) ##(x,y,6,1)
    
    
    ## Calculating Hessian matrix (6)
    hess = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) ##(6,6)
    inv_hess = np.linalg.inv(hess) ##(6,6)
    
    delP = np.ones([6,1],dtype=np.float64)
    iterations = 0
#     while(np.linalg.norm(delP,2) > epsilon): #2-Norm end condition
    while(np.linalg.norm(delP) > epsilon): #Frobenius norm end condition
        
        ## Calculation warp of given image with current guess (1)
        warp_img = cv2.warpAffine(img,warp,(tmply,tmplx)) ##(x,y)
        ## Calculate error image (2)
        err_img = warp_img - template ##(x,y)
        err_img = np.expand_dims(np.expand_dims(err_img,-1),-1) ##(x,y,1,1)
        
        ## Computer other term (7)
        other_term = np.sum(np.sum(np.matmul(steep_desc_trans,err_img),axis=0),axis=0) ##(6,1)
        ## Computing delP (8)
        delP = np.matmul(inv_hess,other_term) ##(6,1)
        
        ## Updating warp (9)
        initP = linearize_warp(warp) ##(6,1)
#         invP = inverse_param(delP) ##(6,1)
        invP = delP
        nextP = compose_param(initP,invP) ##(6,1)
        warp = make_warp(nextP) ##(2,3)
        iterations += 1
        
        print("Iteration %d ; Norm %f" %(iterations,np.linalg.norm(delP,ord='fro')))
        
        if iterations > 5000:
            break
    return warp
    

In [10]:
def lukasKanadeTracker(img1,img2,initialwarp,epsilon, transform='AFFINE'):
#     assert (img1.shape==img2.shape), "Template and Image should be of same shape"
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    return gaussNewton(img1,img2,initialwarp,epsilon,transform)

## Example

In [11]:
# tmpl = cv2.imread("./images/image1.jpg")
# img2 = cv2.imread("./images/image2.jpg")

In [12]:
img1 = cv2.imread("./images/room_left.jpg",0)
img2 = cv2.imread("./images/room_right.jpg",0)

In [13]:
# img = cv2.imread("./images/room_left.jpg")
# img1 = img[50:350,50:450]
# img2 = img[55:355,55:455]

In [14]:
# show_images([("img1",img1),("img2",img2)])

In [15]:
initial_warp = np.float64([[1.0,0.0,0],[0.0,1.0,0]])

In [16]:
# tmpl = img1[50:150,50:150]
# initial_warp = np.float64([[1.0,0.0,-50.0],[0.0,1.0,-50.0]])

In [24]:
# final_warp = lukasKanadeTracker( img2, img1, initial_warp, 0.010, transform='AFFINE')
final_warp = gaussNewton( img2, img1, initial_warp, 0.0000000010, transform='AFFINE')
# final_warp[0,2] -= initial_warp[0,2]
# final_warp[1,2] -= initial_warp[1,2]

Iteration 1 ; Norm 0.025730
Iteration 2 ; Norm 0.025755
Iteration 3 ; Norm 0.025717
Iteration 4 ; Norm 0.025702
Iteration 5 ; Norm 0.025639
Iteration 6 ; Norm 0.025660
Iteration 7 ; Norm 0.025688
Iteration 8 ; Norm 0.025550
Iteration 9 ; Norm 0.025484
Iteration 10 ; Norm 0.025518
Iteration 11 ; Norm 0.025520
Iteration 12 ; Norm 0.025499
Iteration 13 ; Norm 0.025148
Iteration 14 ; Norm 0.024834
Iteration 15 ; Norm 0.024590
Iteration 16 ; Norm 0.024416
Iteration 17 ; Norm 0.024170
Iteration 18 ; Norm 0.024031
Iteration 19 ; Norm 0.023764
Iteration 20 ; Norm 0.023489
Iteration 21 ; Norm 0.023199
Iteration 22 ; Norm 0.022886
Iteration 23 ; Norm 0.022467
Iteration 24 ; Norm 0.022141
Iteration 25 ; Norm 0.021803
Iteration 26 ; Norm 0.021528
Iteration 27 ; Norm 0.021217
Iteration 28 ; Norm 0.020985
Iteration 29 ; Norm 0.020943
Iteration 30 ; Norm 0.020901
Iteration 31 ; Norm 0.020830
Iteration 32 ; Norm 0.020808
Iteration 33 ; Norm 0.020923
Iteration 34 ; Norm 0.021166
Iteration 35 ; Norm 0.0

Iteration 281 ; Norm 0.019344
Iteration 282 ; Norm 0.019463
Iteration 283 ; Norm 0.019496
Iteration 284 ; Norm 0.019596
Iteration 285 ; Norm 0.019675
Iteration 286 ; Norm 0.019739
Iteration 287 ; Norm 0.019792
Iteration 288 ; Norm 0.019904
Iteration 289 ; Norm 0.019997
Iteration 290 ; Norm 0.020040
Iteration 291 ; Norm 0.020121
Iteration 292 ; Norm 0.020207
Iteration 293 ; Norm 0.020243
Iteration 294 ; Norm 0.020291
Iteration 295 ; Norm 0.020321
Iteration 296 ; Norm 0.020325
Iteration 297 ; Norm 0.020367
Iteration 298 ; Norm 0.020446
Iteration 299 ; Norm 0.020469
Iteration 300 ; Norm 0.020531
Iteration 301 ; Norm 0.020550
Iteration 302 ; Norm 0.020611
Iteration 303 ; Norm 0.020641
Iteration 304 ; Norm 0.020724
Iteration 305 ; Norm 0.020829
Iteration 306 ; Norm 0.020974
Iteration 307 ; Norm 0.021105
Iteration 308 ; Norm 0.021200
Iteration 309 ; Norm 0.021337
Iteration 310 ; Norm 0.021462
Iteration 311 ; Norm 0.021573
Iteration 312 ; Norm 0.021668
Iteration 313 ; Norm 0.021782
Iteration 

Iteration 558 ; Norm 0.051165
Iteration 559 ; Norm 0.051301
Iteration 560 ; Norm 0.051436
Iteration 561 ; Norm 0.051570
Iteration 562 ; Norm 0.051771
Iteration 563 ; Norm 0.051925
Iteration 564 ; Norm 0.052146
Iteration 565 ; Norm 0.052294
Iteration 566 ; Norm 0.052549
Iteration 567 ; Norm 0.052814
Iteration 568 ; Norm 0.053083
Iteration 569 ; Norm 0.053281
Iteration 570 ; Norm 0.053592
Iteration 571 ; Norm 0.053783
Iteration 572 ; Norm 0.054130
Iteration 573 ; Norm 0.054326
Iteration 574 ; Norm 0.054655
Iteration 575 ; Norm 0.054931
Iteration 576 ; Norm 0.055232
Iteration 577 ; Norm 0.055526
Iteration 578 ; Norm 0.055847
Iteration 579 ; Norm 0.056148
Iteration 580 ; Norm 0.056493
Iteration 581 ; Norm 0.056749
Iteration 582 ; Norm 0.057079
Iteration 583 ; Norm 0.057383
Iteration 584 ; Norm 0.057706
Iteration 585 ; Norm 0.057939
Iteration 586 ; Norm 0.058268
Iteration 587 ; Norm 0.058562
Iteration 588 ; Norm 0.058835
Iteration 589 ; Norm 0.059120
Iteration 590 ; Norm 0.059439
Iteration 

Iteration 832 ; Norm 0.089872
Iteration 833 ; Norm 0.090119
Iteration 834 ; Norm 0.090425
Iteration 835 ; Norm 0.090667
Iteration 836 ; Norm 0.090950
Iteration 837 ; Norm 0.091199
Iteration 838 ; Norm 0.091420
Iteration 839 ; Norm 0.091678
Iteration 840 ; Norm 0.091906
Iteration 841 ; Norm 0.092151
Iteration 842 ; Norm 0.092353
Iteration 843 ; Norm 0.092521
Iteration 844 ; Norm 0.092706
Iteration 845 ; Norm 0.092905
Iteration 846 ; Norm 0.093120
Iteration 847 ; Norm 0.093327
Iteration 848 ; Norm 0.093494
Iteration 849 ; Norm 0.093639
Iteration 850 ; Norm 0.093792
Iteration 851 ; Norm 0.093930
Iteration 852 ; Norm 0.094010
Iteration 853 ; Norm 0.094118
Iteration 854 ; Norm 0.094198
Iteration 855 ; Norm 0.094300
Iteration 856 ; Norm 0.094383
Iteration 857 ; Norm 0.094464
Iteration 858 ; Norm 0.094520
Iteration 859 ; Norm 0.094582
Iteration 860 ; Norm 0.094627
Iteration 861 ; Norm 0.094646
Iteration 862 ; Norm 0.094675
Iteration 863 ; Norm 0.094696
Iteration 864 ; Norm 0.094709
Iteration 

Iteration 1112 ; Norm 0.144487
Iteration 1113 ; Norm 0.144387
Iteration 1114 ; Norm 0.144255
Iteration 1115 ; Norm 0.144150
Iteration 1116 ; Norm 0.144002
Iteration 1117 ; Norm 0.143816
Iteration 1118 ; Norm 0.143640
Iteration 1119 ; Norm 0.143443
Iteration 1120 ; Norm 0.143294
Iteration 1121 ; Norm 0.143103
Iteration 1122 ; Norm 0.142886
Iteration 1123 ; Norm 0.142685
Iteration 1124 ; Norm 0.142428
Iteration 1125 ; Norm 0.142142
Iteration 1126 ; Norm 0.141900
Iteration 1127 ; Norm 0.141637
Iteration 1128 ; Norm 0.141349
Iteration 1129 ; Norm 0.141014
Iteration 1130 ; Norm 0.140673
Iteration 1131 ; Norm 0.140401
Iteration 1132 ; Norm 0.140097
Iteration 1133 ; Norm 0.139824
Iteration 1134 ; Norm 0.139644
Iteration 1135 ; Norm 0.139400
Iteration 1136 ; Norm 0.139155
Iteration 1137 ; Norm 0.138932
Iteration 1138 ; Norm 0.138686
Iteration 1139 ; Norm 0.138447
Iteration 1140 ; Norm 0.138198
Iteration 1141 ; Norm 0.137971
Iteration 1142 ; Norm 0.137767
Iteration 1143 ; Norm 0.137673
Iteratio

...

Iteration 3832 ; Norm 0.073086
Iteration 3833 ; Norm 0.073400
Iteration 3834 ; Norm 0.073672
Iteration 3835 ; Norm 0.073963
Iteration 3836 ; Norm 0.074251
Iteration 3837 ; Norm 0.074546
Iteration 3838 ; Norm 0.074846
Iteration 3839 ; Norm 0.075156
Iteration 3840 ; Norm 0.075453
Iteration 3841 ; Norm 0.075776
Iteration 3842 ; Norm 0.076106
Iteration 3843 ; Norm 0.076366
Iteration 3844 ; Norm 0.076718
Iteration 3845 ; Norm 0.077055
Iteration 3846 ; Norm 0.077307
Iteration 3847 ; Norm 0.077609
Iteration 3848 ; Norm 0.077939
Iteration 3849 ; Norm 0.078269
Iteration 3850 ; Norm 0.078568
Iteration 3851 ; Norm 0.078855
Iteration 3852 ; Norm 0.079169
Iteration 3853 ; Norm 0.079471
Iteration 3854 ; Norm 0.079784
Iteration 3855 ; Norm 0.080079
Iteration 3856 ; Norm 0.080388
Iteration 3857 ; Norm 0.080685
Iteration 3858 ; Norm 0.080961
Iteration 3859 ; Norm 0.081221
Iteration 3860 ; Norm 0.081525
Iteration 3861 ; Norm 0.081787
Iteration 3862 ; Norm 0.082018
Iteration 3863 ; Norm 0.082263
Iteratio

Iteration 4113 ; Norm 0.063480
Iteration 4114 ; Norm 0.063396
Iteration 4115 ; Norm 0.063298
Iteration 4116 ; Norm 0.063171
Iteration 4117 ; Norm 0.063082
Iteration 4118 ; Norm 0.063014
Iteration 4119 ; Norm 0.062904
Iteration 4120 ; Norm 0.062792
Iteration 4121 ; Norm 0.062692
Iteration 4122 ; Norm 0.062599
Iteration 4123 ; Norm 0.062465
Iteration 4124 ; Norm 0.062376
Iteration 4125 ; Norm 0.062290
Iteration 4126 ; Norm 0.062140
Iteration 4127 ; Norm 0.062060
Iteration 4128 ; Norm 0.061935
Iteration 4129 ; Norm 0.061837
Iteration 4130 ; Norm 0.061711
Iteration 4131 ; Norm 0.061612
Iteration 4132 ; Norm 0.061491
Iteration 4133 ; Norm 0.061425
Iteration 4134 ; Norm 0.061288
Iteration 4135 ; Norm 0.061181
Iteration 4136 ; Norm 0.061072
Iteration 4137 ; Norm 0.060949
Iteration 4138 ; Norm 0.060814
Iteration 4139 ; Norm 0.060719
Iteration 4140 ; Norm 0.060569
Iteration 4141 ; Norm 0.060442
Iteration 4142 ; Norm 0.060319
Iteration 4143 ; Norm 0.060198
Iteration 4144 ; Norm 0.060049
Iteratio

Iteration 4379 ; Norm 0.062686
Iteration 4380 ; Norm 0.062690
Iteration 4381 ; Norm 0.062679
Iteration 4382 ; Norm 0.062664
Iteration 4383 ; Norm 0.062642
Iteration 4384 ; Norm 0.062619
Iteration 4385 ; Norm 0.062589
Iteration 4386 ; Norm 0.062567
Iteration 4387 ; Norm 0.062560
Iteration 4388 ; Norm 0.062509
Iteration 4389 ; Norm 0.062471
Iteration 4390 ; Norm 0.062450
Iteration 4391 ; Norm 0.062401
Iteration 4392 ; Norm 0.062368
Iteration 4393 ; Norm 0.062317
Iteration 4394 ; Norm 0.062250
Iteration 4395 ; Norm 0.062177
Iteration 4396 ; Norm 0.062155
Iteration 4397 ; Norm 0.062073
Iteration 4398 ; Norm 0.062012
Iteration 4399 ; Norm 0.061933
Iteration 4400 ; Norm 0.061893
Iteration 4401 ; Norm 0.061801
Iteration 4402 ; Norm 0.061759
Iteration 4403 ; Norm 0.061696
Iteration 4404 ; Norm 0.061661
Iteration 4405 ; Norm 0.061613
Iteration 4406 ; Norm 0.061551
Iteration 4407 ; Norm 0.061512
Iteration 4408 ; Norm 0.061465
Iteration 4409 ; Norm 0.061411
Iteration 4410 ; Norm 0.061360
Iteratio

Iteration 4646 ; Norm 0.043182
Iteration 4647 ; Norm 0.042931
Iteration 4648 ; Norm 0.042694
Iteration 4649 ; Norm 0.042495
Iteration 4650 ; Norm 0.042232
Iteration 4651 ; Norm 0.042013
Iteration 4652 ; Norm 0.041790
Iteration 4653 ; Norm 0.041560
Iteration 4654 ; Norm 0.041355
Iteration 4655 ; Norm 0.041108
Iteration 4656 ; Norm 0.040920
Iteration 4657 ; Norm 0.040728
Iteration 4658 ; Norm 0.040507
Iteration 4659 ; Norm 0.040313
Iteration 4660 ; Norm 0.040126
Iteration 4661 ; Norm 0.039914
Iteration 4662 ; Norm 0.039753
Iteration 4663 ; Norm 0.039543
Iteration 4664 ; Norm 0.039342
Iteration 4665 ; Norm 0.039154
Iteration 4666 ; Norm 0.038949
Iteration 4667 ; Norm 0.038767
Iteration 4668 ; Norm 0.038585
Iteration 4669 ; Norm 0.038429
Iteration 4670 ; Norm 0.038214
Iteration 4671 ; Norm 0.038062
Iteration 4672 ; Norm 0.037885
Iteration 4673 ; Norm 0.037724
Iteration 4674 ; Norm 0.037544
Iteration 4675 ; Norm 0.037340
Iteration 4676 ; Norm 0.037183
Iteration 4677 ; Norm 0.037002
Iteratio

Iteration 4927 ; Norm 0.026032
Iteration 4928 ; Norm 0.026027
Iteration 4929 ; Norm 0.025954
Iteration 4930 ; Norm 0.025906
Iteration 4931 ; Norm 0.025875
Iteration 4932 ; Norm 0.025827
Iteration 4933 ; Norm 0.025786
Iteration 4934 ; Norm 0.025775
Iteration 4935 ; Norm 0.025731
Iteration 4936 ; Norm 0.025668
Iteration 4937 ; Norm 0.025675
Iteration 4938 ; Norm 0.025675
Iteration 4939 ; Norm 0.025591
Iteration 4940 ; Norm 0.025557
Iteration 4941 ; Norm 0.025514
Iteration 4942 ; Norm 0.025484
Iteration 4943 ; Norm 0.025445
Iteration 4944 ; Norm 0.025396
Iteration 4945 ; Norm 0.025349
Iteration 4946 ; Norm 0.025332
Iteration 4947 ; Norm 0.025289
Iteration 4948 ; Norm 0.025270
Iteration 4949 ; Norm 0.025241
Iteration 4950 ; Norm 0.025194
Iteration 4951 ; Norm 0.025179
Iteration 4952 ; Norm 0.025135
Iteration 4953 ; Norm 0.025072
Iteration 4954 ; Norm 0.025024
Iteration 4955 ; Norm 0.025022
Iteration 4956 ; Norm 0.024943
Iteration 4957 ; Norm 0.024924
Iteration 4958 ; Norm 0.024894
Iteratio

In [25]:
print(final_warp)
print(initial_warp)

[[  1.53828671e+00  -6.27372258e-02  -3.65004986e+02]
 [  3.27952669e-03   9.71622751e-01   1.85896118e+01]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]]


In [26]:
final_warp[0,2] -= initial_warp[0,2]
final_warp[1,2] -= initial_warp[1,2]

In [27]:
warped_img = cv2.warpAffine(img2,final_warp,(img1.shape[1],img1.shape[0]))

In [28]:
final_warp

array([[  1.53828671e+00,  -6.27372258e-02,  -3.65004986e+02],
       [  3.27952669e-03,   9.71622751e-01,   1.85896118e+01]])

In [29]:
show_images([("img1",img1),("img2",img2),("warp",warped_img)])

In [ ]:
warp2 = warped_img.copy()
warp3 = warped_img.copy()

In [ ]:
warp2[warp2==0] = img1[warp2==0]
warp3[warp3==0] = img2[warp3==0]

In [ ]:
show_images([("wp1",warp2),("wp2",warp3)])